# SAM 2024 : TP Requêtes multi-sources pour l'open data

Séances 8 et 9

date du document : 28/03/2024


## BOUZOURINE Hichem - 21319982

Objectif : Comprendre et réaliser des jointures entre deux sources réelles distantes lorsque l'accès aux sources est conforme aux standards de l'*open data*.  

Contexte : On considère deux sources réelles ainsi que des exemples de requêtes pouvant être posées sur chacune d'entre elles.

On vous demande de mettre en oeuvre des scénarios de requêtes **MULTI** sources.

Pour 2024 : Faire l'exercice 3 en priorité




## Préparation

**Copier** ce notebook dans votre drive et l'exécuter dans l'environnement colab de google


In [1]:
!pip install SPARQLWrapper

Defaulting to user installation because normal site-packages is not writeable


### Manipuler le résultat d'une requête
Fonctions **tableauResultat** pour extraire  le résultat d'une requête

In [2]:
import pandas as pd

#pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Déterminer la liste des noms d'attributs du résultat, en tenant compte des attributs optionnels.
def schemaResultat(resultats, limit=100):
  d = {}
  for elt in resultats[:limit]:
    for (pos, nom) in enumerate(elt):
      sum_pos, nb = d.get(nom, (0, 0))
      d[nom] = (sum_pos + pos, nb + 1)
  mean_pos_by_name = [(sum_pos / nb, nom) for (nom, (sum_pos, nb)) in d.items()]
  colonnes = [nom for (mean_pos, nom) in sorted(mean_pos_by_name)]
  return colonnes

def tableauResultat(resultats, limit=100, colonnes=None):
  if colonnes is None :
    colonnes = schemaResultat(resultats, limit)
  tableau = []
  for elt in resultats[:limit]:
    row = {name:v.value for name, v in elt.items()}
    tableau.append([row.get(n, "") for n in colonnes])
  return tableau

print ("fonctions définies")

fonctions définies


Afficher le résultat

In [3]:
def afficherResultat(resultats, limit=100):
  print(len(resultats), "éléments")
  colonnes = schemaResultat(resultats, limit)
  print("schéma: ", colonnes)
  result = tableauResultat(resultats, limit, colonnes)
  return pd.DataFrame(result, columns = colonnes)

print ("fonctions définies")

fonctions définies


### Importer le pilote pour accéder aux sources

In [4]:
from SPARQLWrapper import SPARQLWrapper2
print("pilote importé")

pilote importé


### Préparer l'accès à la source BNF

In [5]:
# Connexion à la BNF
BNF = SPARQLWrapper2("http://data.bnf.fr/sparql")

BNF_prefix = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>
"""
print(BNF_prefix)


PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>



### Préparer l'accès à la source DBPEDIA

In [6]:
# Connexion à DBPEDIA France.
DBPEDIA = SPARQLWrapper2("http://fr.dbpedia.org/sparql")

DBPEDIA_prefix =  """
PREFIX dc:   <http://purl.org/dc/terms/>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX dbo:  <http://dbpedia.org/ontology/>

# Dans cette source, les noms de propriétés sont en français
PREFIX dbp:  <http://fr.dbpedia.org/property/>
"""
# print(DBPEDIA_prefix)

## Source 1 : La bibliothèque BNF

### BNF P1: Le nom des personnes
On suppose les relations :
*    **CrééPar**(ouvrage, auteur)
*    **Nom**(auteur, nom)

En SQL, la requête s'exprimerait ainsi:

> select distinct n.nom

> from CrééPar c, Nom n

> where c.auteur = n.auteur

In [7]:
requete = BNF_prefix + """
SELECT distinct ?nom
WHERE {
  ?ouvrage dc:creator ?auteur .       # un ouvrage est créé par au moins un auteur
  ?auteur foaf:name ?nom              # un auteur a un nom
}
LIMIT 5
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
for r in tableauResultat(resultats):
  print(r)

['Ahmed Ben Brihmat']
['Constantin Roussel']
['François-Marie-Stéphane Guillot']
['Paul Émilien Félix Defontaine']
['François Joseph Ferdinand Marchal']


### BNF T1: liste de thèmes
#### Les *themes* des oeuvres dont l'auteur est référencé dans dbpedia france

En SQL, cette requête pourrait s'exprimer sur les tables suivantes

**créé_par** (livre, auteur)

**type_d_expression** (livre, type_expre)  -- exple de tpye d'expression: textuelle

**écrit_en** (livre, langue)

**sujet** (livre, thème)

**auteur_équivalent** (auteur, auteur_source2)

et une condition SQL serait : WHERE auteur_source2 like 'http://fr.dbpedia.org%'

In [8]:
requete = BNF_prefix + """
SELECT distinct ?theme
WHERE {# A compléter


  # une oeuvre est associée a (au moins) un auteur
  ?livre dc:creator ?auteur .

  # un livre est un ouvrage dont la forme d'expression est textuelle (te)
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # livre écrit en langue française
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a au moins un theme
  ?livre bnf:subject ?theme.

  # l'auteur existe également dans une autre source,
  # on dit qu'il est "aligné" avec un autre instance du même individu.
  ?auteur owl:sameAs ?auteurDbpedia .

  # la référence externe "pointe" vers dbpedia
  FILTER ( regex(?auteurDbpedia, "http://fr.dbpedia.org")  )

}
ORDER BY ?theme
LIMIT 20
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

# Exemple:
# Informatique
# Langues
# Philosophie

20 éléments
schéma:  ['theme']


,theme
0,Administration publique
1,Anthropologie. Ethnologie
2,Architecture
3,Archéologie. Préhistoire. Histoire ancienne
4,Art
5,Art et science militaires
6,Arts du spectacle
7,Arts graphiques
8,Astronomie
9,Audiovisuel


###  BNF A1 : une liste d'auteurs

A1 : Liste des auteurs de livres écrits en français et nés pendant une certaine période, par exemple au 20eme siècle. Afficher le nom de l'auteur, son année et lieu de naissance.



In [9]:
requete = BNF_prefix + """
SELECT distinct ?nom  ?naissance ?lieu
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom
?auteur foaf:name ?nom .

# un auteur a une année et un lieu de naissance
?auteur bio:birth ?naissance.
?auteur voc2:placeOfBirth ?lieu

# on sélectionne les années de naissance dans l'intervalle [1900, 2000]
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer
     && xsd:integer (?naissance) < '2000'^^xsd:integer )
}
#ORDER BY ?nom
LIMIT 20
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

20 éléments
schéma:  ['nom', 'naissance', 'lieu']


,nom,naissance,lieu
0,Enrichetta Leospo,1947,"Turin, Italie"
1,Jean Taffanel,1917,Mailhac (Aude)
2,Marie-Thérèse Laureilhe,1913,Paris
3,Stefan Priacel,1904,Wroclaw (Pologne)
4,Albert Meister,1927,Delsberg (Suisse)
5,André Tuilier,1921,Paris (France)
6,Michèle Faivre-Jussiaux,1947,Pays de Montbéliard (Doubs)
7,Alexandre Popovic,1931,"Belgrade (Yougoslavie, aujourd'hui Serbie)"
8,Marthe Philipp,1922,"Essen, RFA"
9,Germaine Sneyers,1907,Schaerbeek (Belgique)


###  BNF A2 : les auteurs de la BNF référencés dans dbpedia
Les auteurs ayant un identifiant externe pointant vers dbpedia


In [10]:
requete = BNF_prefix + """
SELECT distinct ?nom  ?naissance ?auteurDbpedia
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# naissance dans l'intervalle [1900, 2000] et la référence externe "pointe" vers dbpedia
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer
     && xsd:integer (?naissance) < '2000'^^xsd:integer
     && regex(?auteurDbpedia, "http://fr.dbpedia.org")
     )
}
ORDER BY desc(?naissance)
LIMIT 10
"""

BNF.setQuery(requete)

resultats = BNF.query().bindings
afficherResultat(resultats)

10 éléments
schéma:  ['nom', 'naissance', 'auteurDbpedia']


,nom,naissance,auteurDbpedia
0,Vickie Gendreau,1989,http://fr.dbpedia.org/resource/Vickie_Gendreau
1,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
2,Geneviève Castrée,1981,http://fr.dbpedia.org/resource/Genevi%C3%A8ve_Castr%C3%A9e
3,Isabelle Périer,1978,http://fr.dbpedia.org/resource/Isabelle_P%C3%A9rier
4,Nathanaël Dupré La Tour,1977,http://fr.dbpedia.org/resource/Nathana%C3%ABl_Dupr%C3%A9_La_Tour
5,Coralie Delaume,1976,http://fr.dbpedia.org/resource/Coralie_Delaume
6,Claire Guézengar,1972,http://fr.dbpedia.org/resource/Claire_Guezengar
7,Philippe Nassif,1971,http://fr.dbpedia.org/resource/Philippe_Nassif
8,Fatou Cissé,1971,http://fr.dbpedia.org/resource/Fatou_Fanny-Ciss%C3%A9
9,Benoït Violier,1971,http://fr.dbpedia.org/resource/Beno%C3%AEt_Violier


### BNF A3 : Les auteurs d'une oeuvre philosophique

Voir T1 pour la liste des thèmes possibles autres que la philosophie.

On se limite aux oeuvres écrite en français et sont l'auteur est référencé dans dbpedia france.

In [11]:
requete = BNF_prefix + """
SELECT distinct ?nom ?auteurDbpedia ?type_ouvrage
WHERE {

  ?livre dc:creator ?auteur .

  # un livre est un ouvrage textuel
  # (revoir la formulation de ce triplet...)
  # ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org")
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia

LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats, 5)

20 éléments
schéma:  ['nom', 'auteurDbpedia']


,nom,auteurDbpedia
0,Alexis Carrel,http://fr.dbpedia.org/resource/Alexis_Carrel
1,Antoine de Rivarol,http://fr.dbpedia.org/resource/Antoine_de_Rivarol
2,Bernard-Henri Lévy,http://fr.dbpedia.org/resource/Bernard-Henri_L%C3%A9vy
3,Bernard de Fontenelle,http://fr.dbpedia.org/resource/Bernard_Le_Bouyer_de_Fontenelle
4,Claude Guillon,http://fr.dbpedia.org/resource/Claude_Guillon_%28%C3%A9crivain%29


###  BNF L1 : Les livres avec leurs auteurs

Les livres écrit en français au 21eme siècle,  référencés dans wikidata et dont l'auteur est référencé dans dbpedia

In [12]:
requete = BNF_prefix + """
SELECT distinct ?annee ?nom ?titre ?auteurDbpedia
WHERE {

  ?livre dc:creator ?auteur .

  # l'année de parution du livre
  ?livre dc:date ?annee .

  # le titre du livre
  ?livre dc:title ?titre .

  # un livre est un ouvrage textuel
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le nom de l'auteur
  ?auteur foaf:name ?nom.

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER (
       xsd:integer (?annee) > '2000'^^xsd:integer
    && xsd:integer (?annee) < '2099'^^xsd:integer
    && regex(?livreWiki, "http://wikidata.org")
    &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
#ORDER BY desc(?annee)

LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats)

20 éléments
schéma:  ['annee', 'nom', 'titre', 'auteurDbpedia']


,annee,nom,titre,auteurDbpedia
0,2006,Jonathan Littell,Les Bienveillantes,http://fr.dbpedia.org/resource/Jonathan_Littell
1,2001,Claude Simon,Le tramway,http://fr.dbpedia.org/resource/Claude_Simon
2,2002,André Gide,Le ramier,http://fr.dbpedia.org/resource/Andr%C3%A9_Gide
3,2001,Yves Bonnefoy,Les planches courbes,http://fr.dbpedia.org/resource/Yves_Bonnefoy
4,2003,J. M. G. Le Clézio,Révolutions,http://fr.dbpedia.org/resource/J._M._G._Le_Cl%C3%A9zio
5,2001,Catherine Millet,La vie sexuelle de Catherine M.,http://fr.dbpedia.org/resource/Catherine_Millet
6,2001,Michel Houellebecq,Plateforme,http://fr.dbpedia.org/resource/Michel_Houellebecq
7,2005,Michel Onfray,Traité d'athéologie,http://fr.dbpedia.org/resource/Michel_Onfray
8,2006,Pascal Quignard,Villa Amalia,http://fr.dbpedia.org/resource/Pascal_Quignard
9,2007,Nathacha Appanah,Le dernier frère,http://fr.dbpedia.org/resource/Nathacha_Appanah


## Source 2 : DBPedia France

### D1 : Informations sur une personne dont le nom contient certains mots.
Afficher le nom, nationalité, profession, date et lieu de naissance

In [13]:
requete_parametree = DBPEDIA_prefix + """
  SELECT ?p ?name ?nationalité ?profession year(xsd:dateTime(?naissance)) as ?naissance  ?lieu
  WHERE {
    ?p a dbo:Person .
    ?p foaf:name ?name .
    ?p dbo:birthDate ?naissance
    optional {?p dbp:nationalité ?nationalité}
    optional {?p dbp:profession ?profession}
    optional {?p dbp:lieuDeNaissance ?lieu}

  FILTER (regex(?p, "{MOTIF}", "i") )
  }
  LIMIT 10
"""

# l'expression régulière du nom à retrouver
motif = "Victor.*Hug"

requete = requete_parametree.replace("{MOTIF}", motif)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['p', 'name', 'nationalité', 'profession', 'naissance', 'lieu']


,p,name,nationalité,profession,naissance,lieu
0,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
1,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
2,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
3,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
4,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
5,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
6,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
7,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
8,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/New_York
9,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/États-Unis


### D2 : les infos détaillées sur une certaine personne

In [14]:
requete_parametree = DBPEDIA_prefix + """
  SELECT distinct  ?name ?nationalite ?interet ?idee
  WHERE {
    ?pers foaf:name ?name
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
  }
  order by ?interet ?idee
  LIMIT 20
"""
# On fixe une certaine personne
personne = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#personne = 'http://fr.dbpedia.org/resource/Jean-Paul_Sartre'

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", personne)

# print("la requête est :", requete)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

3 éléments
schéma:  ['name', 'interet', 'idee']


,name,interet,idee
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Philosophie,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"


### D3 : la liste des oeuvres principales d'une personne donnée

In [15]:
requete_parametree = DBPEDIA_prefix +  """
    SELECT distinct ?name ?oeuvre
    WHERE {
      ?pers foaf:name ?name
      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
}
order by ?oeuvre
LIMIT 10
"""
# On fixe une certaine personne
personne = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'

requete = requete_parametree.replace("{PERSONNE}", personne)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

5 éléments
schéma:  ['name', 'oeuvre']


,name,oeuvre
0,Simone de Beauvoir,L'Invitée
1,Simone de Beauvoir,La Force de l'âge
2,Simone de Beauvoir,Le Deuxième Sexe
3,Simone de Beauvoir,Les Mandarins
4,Simone de Beauvoir,Mémoires d'une jeune fille rangée


### D4 : les influenceurs d'une personne donnée

In [16]:
requete_parametree = DBPEDIA_prefix + """
   SELECT distinct ?name ?influenceur
    WHERE {
      ?pers foaf:name ?name
      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
}
order by ?influenceur
LIMIT 10
"""

# On fixe une certaine personne
PERSONNE = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#PERSONNE = 'http://fr.dbpedia.org/resource/Judith_Butler'

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
print(len(res), 'resultats')
afficherResultat(res)

10 resultats
10 éléments
schéma:  ['name', 'influenceur']


,name,influenceur
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Alfred_Adler
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Edmund_Husserl
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Emmanuel_Kant
3,Simone de Beauvoir,http://fr.dbpedia.org/resource/François_Poullain_de_La_Barre
4,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Engels
5,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Nietzsche
6,Simone de Beauvoir,http://fr.dbpedia.org/resource/Georg_Wilhelm_Friedrich_Hegel
7,Simone de Beauvoir,http://fr.dbpedia.org/resource/Jean-Paul_Sartre
8,Simone de Beauvoir,http://fr.dbpedia.org/resource/Karl_Marx
9,Simone de Beauvoir,http://fr.dbpedia.org/resource/Martin_Heidegger


### D5 : Les auteurs nés en 2001
Utiliser la fonction year pour extraire l'année de naissance à partir d'une date de naissance.

In [17]:
requete_parametree = DBPEDIA_prefix + """
    SELECT distinct ?p ?name year(xsd:dateTime(?naissance)) as ?naissance
    WHERE {
      ?p a dbo:Person .
      ?p foaf:name ?name .
      ?p dbo:birthDate ?naissance
      FILTER( regex(?naissance, "{ANNEE}") && STRLEN(?name) != 0)
    }
    order by ?p
    LIMIT 10
"""

année = 2001
requete = requete_parametree.replace("{ANNEE}", str(année))

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['p', 'name', 'naissance']


,p,name,naissance
0,http://fr.dbpedia.org/resource/Abraham_Attah,Abraham Attah,2001
1,http://fr.dbpedia.org/resource/Adam_Siao_Him_Fa,Adam Siao Him Fa,2001
2,http://fr.dbpedia.org/resource/Adrien_Warion,Adrien Warion,2001
3,http://fr.dbpedia.org/resource/Adèle_Castillon,Adèle Castillon,2001
4,http://fr.dbpedia.org/resource/Ai_Ogura,Ai Ogura,2001
5,http://fr.dbpedia.org/resource/Aiko_de_Toshi,Aiko,2001
6,http://fr.dbpedia.org/resource/Aleksandr_Nikichine,Aleksandr Nikichine,2001
7,http://fr.dbpedia.org/resource/Aleksandr_Smolyar,Aleksandr Smolyar,2001
8,http://fr.dbpedia.org/resource/Aleksej_Pokuševski,Aleksej Pokuševski,2001
9,http://fr.dbpedia.org/resource/Alessandro_Michieletto,Alessandro Michieletto,2001


### D6 : La liste de tous les intérets principaux


In [18]:
requete = DBPEDIA_prefix + """
    SELECT distinct ?interet
    WHERE {
      ?personne dbo:mainInterest ?o .
      ?o rdfs:label ?interet .

      FILTER (langMatches( lang(?interet), "FR" ) )
}
order by ?interet
LIMIT 10
"""

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['interet']


,interet
0,Activisme politique
1,Afrique
2,Agnosticisme
3,Alchimie
4,Amour
5,Amour platonique
6,Angélologie
7,Anthropologie
8,Anthropologie linguistique
9,Anthropologie philosophique


### D7 la liste des philosophes dans Dbpedia

In [19]:
import time

requete = DBPEDIA_prefix + """

SELECT distinct  ?name ?p
WHERE {
  ?p rdf:type dbo:Philosopher .
  ?p foaf:name ?name
}
order by ?name
limit 10
"""

start_time = time.time()

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings

duree = int ( (time.time() - start_time) * 1000)
print(f"durée {duree} ms")

afficherResultat(res)


durée 191 ms
10 éléments
schéma:  ['name', 'p']


,name,p
0,,http://fr.dbpedia.org/resource/Œuvre_de_Friedrich_Nietzsche
1,,http://fr.dbpedia.org/resource/Abu_'Ali_Al-Jubbâ'i
2,,http://fr.dbpedia.org/resource/Abu_al-Hudhayl
3,,http://fr.dbpedia.org/resource/Abū_Hāshīm_al-Jubbā'ī
4,,http://fr.dbpedia.org/resource/Adam_Smith
5,,http://fr.dbpedia.org/resource/Aimé_Forest
6,,http://fr.dbpedia.org/resource/Al-Fârâbî
7,,http://fr.dbpedia.org/resource/Al-Ghazali
8,,http://fr.dbpedia.org/resource/Alain_(philosophe)
9,,http://fr.dbpedia.org/resource/Alfred_Tarski


# Exercice 1 : Requêtes réparties entre les 2 sources BNF et DBPedia

### J1: Jointure A3 avec D2 par boucles imbriquées
Proposer une solution pour calculer la jointure entre A3 sur le site BNF et D2 sur le site DBPedia.
Pour chaque livre de afficher son titre et son année, le nom de l'auteur, sa nationalité et la liste de ses idées remarquables.

In [20]:
requete = (
    BNF_prefix
    + """
SELECT distinct ?livre ?nom ?naissance ?auteurDbpedia
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# naissance dans l'intervalle [1900, 2000] et la référence externe "pointe" vers dbpedia
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer 
     && xsd:integer (?naissance) < '2000'^^xsd:integer 
     && regex(?auteurDbpedia, "http://fr.dbpedia.org") 
     )
}
ORDER BY desc(?naissance)
LIMIT 1000
"""
)

BNF.setQuery(requete)

res_A3 = BNF.query().bindings
df_A3 = pd.DataFrame(res_A3).map(lambda x: x.value)

In [21]:
df_A3

,livre,nom,naissance,auteurDbpedia
0,http://data.bnf.fr/temp-work/74939adb91e2e79ddf037e7bb72be966/#about,Vickie Gendreau,1989,http://fr.dbpedia.org/resource/Vickie_Gendreau
1,http://data.bnf.fr/temp-work/3ccbde1f01fcb26e705a42921d161156/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
2,http://data.bnf.fr/temp-work/679ee62bf2e6d1965c424dc8f48f3b98/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
3,http://data.bnf.fr/temp-work/2be1462996455b4d11ec261812ed4497/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
4,http://data.bnf.fr/temp-work/dda86386c8a378808e05b81b53424a1a/#about,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
...,...,...,...,...
995,http://data.bnf.fr/temp-work/4e29441740b8eea16f7b2c65ee68685a/#about,Gérard Salem,1946,http://fr.dbpedia.org/resource/G%C3%A9rard_Salem
996,http://data.bnf.fr/temp-work/1dff8b9d989330680fc4717b3f69b32d/#about,Gérard Salem,1946,http://fr.dbpedia.org/resource/G%C3%A9rard_Salem
997,http://data.bnf.fr/temp-work/686f8d505a57396c2dc6aa73c18bd58b/#about,Gérard Salem,1946,http://fr.dbpedia.org/resource/G%C3%A9rard_Salem
998,http://data.bnf.fr/temp-work/7877bad96453a4fcfec6147dd7300eda/#about,Gérard Salem,1946,http://fr.dbpedia.org/resource/G%C3%A9rard_Salem


In [22]:
personnes = df_A3["auteurDbpedia"].unique()
len(personnes)

168

In [23]:

requete_parametree = (
    DBPEDIA_prefix
    + """
  SELECT distinct ?name ?nationalite ?interet ?idee
  WHERE { 
    ?pers foaf:name ?name 
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

#      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
#      optional {?pers  dbo:influencedBy ?influenceur}
    FILTER ( ?pers = <{PERSONNE}> ) 

  }
  order by ?p
  LIMIT 10
"""
)

    # FILTER ( ?pers = <{PERSONNE}> ) 

res = []
for personne in personnes:
    DBPEDIA.setQuery(requete_parametree.replace("{PERSONNE}", personne))
    D2 = DBPEDIA.query().bindings
    res += D2

afficherResultat(res)


49 éléments
schéma:  ['name', 'nationalite']


,name,nationalite
0,,
1,Philippe Nassif,
2,,
3,Franck Krebs,
4,,
5,Thierry Aprile,
6,,
7,Emmanuel Jouanne,
8,,
9,Bruno Fecteau,


### J2 : Pour une année N afficher la liste des personnes qui sont nées l'année N et qui existent à la fois dans la BNF et dans DBPedia.
Proposer une solution par fusion de deux résultats triés

In [24]:
# A compléter
annee = 1923

req1 = (
    BNF_prefix
    + """
SELECT distinct ?nom ?auteurDbpedia ?naissance
WHERE {
?auteur foaf:name ?nom ; bio:birth ?naissance .
?auteur owl:sameAs ?auteurDbpedia .
FILTER (regex(?auteurDbpedia, "http://fr.dbpedia.org")
        && xsd:integer(?naissance) = '{ANNEE}'^^xsd:integer )
}
ORDER BY desc(?naissance)
LIMIT 1000
"""
)

BNF.setQuery(req1.replace("{ANNEE}", str(annee)))
res1 = BNF.query().bindings
df1 = pd.DataFrame(res1).map(lambda x: x.value)

afficherResultat(res1)


123 éléments
schéma:  ['nom', 'auteurDbpedia', 'naissance']


,nom,auteurDbpedia,naissance
0,Robert de La Rochefoucauld,http://fr.dbpedia.org/resource/Robert_de_La_Rochefoucauld,1923
1,Paul Cuvelier,http://fr.dbpedia.org/resource/Paul_Cuvelier,1923
2,Jean Deprun,http://fr.dbpedia.org/resource/Jean_Deprun,1923
3,Denise Levertov,http://fr.dbpedia.org/resource/Denise_Levertov,1923
4,Meng-Hua Ho,http://fr.dbpedia.org/resource/Ho_Meng-hua,1923
...,...,...,...
95,Renato Polese,http://fr.dbpedia.org/resource/Renato_Polese,1923
96,Martha Schlamme,http://fr.dbpedia.org/resource/Martha_Schlamme,1923
97,Seydou Keita,http://fr.dbpedia.org/resource/Seydou_Ke%C3%AFta_%28photographe%29,1923
98,Jean Darnel,http://fr.dbpedia.org/resource/Jean_Darnel,1923


In [32]:
req2 = (
    DBPEDIA_prefix
    + """
  SELECT distinct ?p year(xsd:dateTime(?naissance)) as ?naissance
  WHERE { 
    ?p a dbo:Person .
    ?p dbo:birthDate ?naissance
    FILTER ( regex (?naissance, '{ANNEE}') )
  }
  ORDER BY desc(?naissance)
  LIMIT 10000
"""
)

DBPEDIA.setQuery(req2.replace("{ANNEE}", str(annee)))
res2 = DBPEDIA.query().bindings
df2 = pd.DataFrame(res2).map(lambda x: x.value)

afficherResultat(res2)


1437 éléments
schéma:  ['p', 'naissance']


,p,naissance
0,http://fr.dbpedia.org/resource/Ștefan_Stănculescu,1923
1,http://fr.dbpedia.org/resource/Aage_Larsen,1923
2,http://fr.dbpedia.org/resource/Aaron_Antonovsky,1923
3,http://fr.dbpedia.org/resource/Abdelaziz_Benabdallah,1923
4,http://fr.dbpedia.org/resource/Abdelaziz_Bouraoui,1923
...,...,...
95,http://fr.dbpedia.org/resource/Anne_Shelton_(chanteuse),1923
96,http://fr.dbpedia.org/resource/Anne_de_Bourbon-Parme,1923
97,http://fr.dbpedia.org/resource/Antal_Bánkúti,1923
98,http://fr.dbpedia.org/resource/Anthony_Joseph_Bevilacqua,1923


In [26]:
df1.merge(df2, left_on="auteurDbpedia", right_on="p")

,nom,auteurDbpedia,naissance_x,p,naissance_y
0,Paul Cuvelier,http://fr.dbpedia.org/resource/Paul_Cuvelier,1923,http://fr.dbpedia.org/resource/Paul_Cuvelier,1923
1,Denise Levertov,http://fr.dbpedia.org/resource/Denise_Levertov,1923,http://fr.dbpedia.org/resource/Denise_Levertov,1923
2,Meng-Hua Ho,http://fr.dbpedia.org/resource/Ho_Meng-hua,1923,http://fr.dbpedia.org/resource/Ho_Meng-hua,1923
3,Robert Soulat,http://fr.dbpedia.org/resource/Robert_Soulat,1923,http://fr.dbpedia.org/resource/Robert_Soulat,1923
4,Cyril M. Kornbluth,http://fr.dbpedia.org/resource/Cyril_M._Kornbluth,1923,http://fr.dbpedia.org/resource/Cyril_M._Kornbluth,1923
5,Simon Eisner,http://fr.dbpedia.org/resource/Cyril_M._Kornbluth,1923,http://fr.dbpedia.org/resource/Cyril_M._Kornbluth,1923
6,Kildare Dobbs,http://fr.dbpedia.org/resource/Kildare_Dobbs,1923,http://fr.dbpedia.org/resource/Kildare_Dobbs,1923
7,Richard Wyler,http://fr.dbpedia.org/resource/Richard_Stapley,1923,http://fr.dbpedia.org/resource/Richard_Stapley,1923
8,Piera Aulagnier,http://fr.dbpedia.org/resource/Piera_Aulagnier,1923,http://fr.dbpedia.org/resource/Piera_Aulagnier,1923
9,Urbano Tavares Rodrigues,http://fr.dbpedia.org/resource/Urbano_Tavares_Rodrigues,1923,http://fr.dbpedia.org/resource/Urbano_Tavares_Rodrigues,1923


### J3 : Quels sont les auteurs d'une oeuvre dont le thème est philosophie (dans la BNF) et qui sont référencés comme étant des philosophes dans DBpedia ?

In [27]:
# A compléter

# Exercice 2 : Accès aux données par des requêtes de sélections
L'objectif de cet exercice est d'appréhender la situation où l'accès aux données est contraint (voir la notion de [linked data fragment](https://linkeddatafragments.org/concept/)). Le point d'accès n'autorise que des requêtes simples (donc peut coûteuses à traiter) qui ne contiennent que des **sélections**. La clause where d'une requête ne peut contenir qu'**un seul** motif ayant 1 ou 2 variables. Les motifs autorisés sont donc parmi :
*   ?a *propriété* ?b *(cette requête a deux variables)*
*   *A* *propriété* ?b  *(cette requête a une seule variable)*
*   ?a *propriété* *B*  

où ?a et ?b sont des variables et *A*, *B* et *propriété* sont des valeurs fixées avant de poser la requête.

Ainsi, toutes les jointures doivent être exprimées par le client qui accède aux données : vous devez "programmer" toutes les jointures en python.
Pour les requêtes D1 et D6 proposer une solution respectant ce mode d'accès.

In [28]:
# pip install tabulate

In [29]:
# # à compléter
# from SPARQLWrapper import SPARQLWrapper, JSON
# from tabulate import tabulate
# # Définir l'URL du point d'accès DBpedia
# DBPEDIA_prefix = "http://dbpedia.org/sparql"

# # Définir l'objet SPARQLWrapper pour le point d'accès DBpedia
# DBPEDIA = SPARQLWrapper(DBPEDIA_prefix)
# DBPEDIA.setReturnFormat(JSON)

# # Fonction pour effectuer une requête simple et récupérer les résultats
# def effectuer_requete(requete):
#     DBPEDIA.setQuery(requete)
#     res = DBPEDIA.query().convert()
#     return res['results']['bindings']

# # Requête principale pour récupérer les informations de base sur les personnes
# requete_principale = """
# SELECT ?p ?name ?naissance
# WHERE {
#   ?p a dbo:Person .
#   ?p foaf:name ?name .
#   ?p dbo:birthDate ?naissance
#   FILTER (regex(?name, "{MOTIF}", "i"))
# }
# LIMIT 10
# """

# # Remplacer le motif dans la requête principale
# motif = "Victor.*Hug"
# requete_principale = requete_principale.replace("{MOTIF}", motif)

# # Effectuer la requête principale en utilisant la fonction effectuer_requete
# resultats_principaux = effectuer_requete(requete_principale)

# # Liste pour stocker les résultats complets
# resultats_complets = []

# # Parcourir les résultats principaux pour récupérer les informations supplémentaires
# for resultat in resultats_principaux:
#     personne = {
#         'p': resultat['p']['value'],
#         'name': resultat['name']['value'],
#         'naissance': resultat['naissance']['value']
#     }
    
#     # Requêtes supplémentaires pour récupérer la nationalité, la profession et le lieu de naissance
#     requete_nationalite = f"""
#     SELECT ?nationalité WHERE {{
#       <{personne['p']}> dbp:nationalité ?nationalité
#     }}
#     """
#     requete_profession = f"""
#     SELECT ?profession WHERE {{
#       <{personne['p']}> dbp:profession ?profession
#     }}
#     """
#     requete_lieu_naissance = f"""
#     SELECT ?lieu WHERE {{
#       <{personne['p']}> dbp:lieuDeNaissance ?lieu
#     }}
#     """
    
#     # Exécuter les requêtes supplémentaires et ajouter les informations au dictionnaire de la personne
#     nationalite = effectuer_requete(requete_nationalite)
#     profession = effectuer_requete(requete_profession)
#     lieu_naissance = effectuer_requete(requete_lieu_naissance)
    
#     if nationalite:
#         personne['nationalité'] = nationalite[0]['nationalité']['value']
#     else:
#         personne['nationalité'] = None
        
#     if profession:
#         personne['profession'] = profession[0]['profession']['value']
#     else:
#         personne['profession'] = None
        
#     if lieu_naissance:
#         personne['lieu'] = lieu_naissance[0]['lieu']['value']
#     else:
#         personne['lieu'] = None
        
#     resultats_complets.append(personne)

# # Afficher les résultats complets sous forme tabulaire
# print(tabulate(resultats_complets, headers="keys", tablefmt="grid"))

# Exercice 3 : Accès par morceau au résultat d'une requête
1) Un utilisateur veut accéder aux données de manière progressive. Proposer une solution pour obtenir le resultat d'une requete par morceau de N tuples, en utilisant les mots-clés OFFSET et LIMIT. Appliquer cela pour lire le résultat de la requête qui affiche le nom des personnes existant dans dbpedia.
Etudier l'influence de l'offset et de la taille du résultat sur le temps de réponse de la requête.



In [30]:
# Exemple d'utilisation de OFFSET et LIMIT

# Compléter cet exemple


import time

requete = DBPEDIA_prefix + """

SELECT distinct ?p ?name
WHERE {
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
  FILTER(  STRLEN(?name) != 0)
}
# ORDER BY ?p ?name
offset 10000 limit 10
"""

start_time = time.time()
# exécution de la requête
DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
duree = int ( (time.time() - start_time) * 1000)
print(f"la requête dure {duree} ms")

afficherResultat(res)

la requête dure 108 ms
10 éléments
schéma:  ['p', 'name']


,p,name
0,http://fr.dbpedia.org/resource/Alina_Alexandra_Dumitru,Alina Alexandra Dumitru
1,http://fr.dbpedia.org/resource/Alina_Astafei,Alina Astafei
2,http://fr.dbpedia.org/resource/Alina_Azario,Alina Azario
3,http://fr.dbpedia.org/resource/Alina_Berchakova,Alina Berchakova
4,http://fr.dbpedia.org/resource/Alina_Berezhna,Alina Berezhna
5,http://fr.dbpedia.org/resource/Alina_Dumitrescu,Alina Dumitrescu
6,http://fr.dbpedia.org/resource/Alina_Elizarova,Alina Elizarova
7,http://fr.dbpedia.org/resource/Alina_Freund,Alina Freund
8,http://fr.dbpedia.org/resource/Alina_Fyodorova,Alina Fyodorova
9,http://fr.dbpedia.org/resource/Alina_Goreac,Alina Goreac


In [31]:
import time

# Définition de la requête SPARQL avec OFFSET et LIMIT
requete = DBPEDIA_prefix + """
SELECT distinct ?p ?name
WHERE {
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
  FILTER(STRLEN(?name) != 0)
}
ORDER BY ?p ?name
OFFSET {OFFSET} LIMIT {LIMIT}
"""

# Paramètres pour contrôler le nombre de tuples récupérés à chaque étape
OFFSET = 0  # Début à partir de la première ligne
LIMIT = 10  # Nombre de tuples à récupérer à chaque étape

start_time = time.time()

# Boucle pour lire les données progressivement
while True:
    # Modification de la requête avec les valeurs d'OFFSET et de LIMIT
    requete_modifiee = requete.replace("{OFFSET}", str(OFFSET)).replace("{LIMIT}", str(LIMIT))

    # Exécution de la requête
    DBPEDIA.setQuery(requete_modifiee)
    res = DBPEDIA.query().bindings
    
    # Affichage des résultats
    afficherResultat(res)
    
    # Vérification s'il y a plus de données à récupérer
    if len(res) < LIMIT:
        break
    
    # Mise à jour de l'OFFSET pour la prochaine itération
    OFFSET += LIMIT

# Calcul de la durée totale d'exécution de la requête
duree = int((time.time() - start_time) * 1000)
print(f"La requête a duré {duree} ms")


10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']
10 éléments
schéma:  ['p', 'name']


KeyboardInterrupt: 

2) Définir une fonction traiteRequete(requete) qui retourne un generateur sur une requete de telle sorte que la requete soit lue par morceaux de N tuples et que seuls les morceaux permettant de produire les M lignes demandées par l'utlisateur soit lus. On ne connait pas à l'avance le nombre M de lignes que va lire l'utilisateur. On a M > N.
Indication voir le mot clé python **yield** pour retourner un générateur.

In [33]:
requete = DBPEDIA_prefix + """
SELECT distinct ?name ?p
WHERE {
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
  FILTER(  STRLEN(?name) != 0)
}
OFFSET 0 LIMIT 10
"""

# définition d'un générateur
def traiteRequete(requete, N, M):
    offset = 0
    total_read = 0
    while total_read < M:
        # Update the query with the current offset
        query = requete + f" OFFSET {offset} LIMIT {N}"
        
        # Execute the query
        DBPEDIA.setQuery(query)
        results = DBPEDIA.query().convert()
        result_count = len(results['results']['bindings'])
        
        # Yield each result
        for r in results['results']['bindings']:
            yield r
            total_read += 1
            if total_read >= M:
                break
                
        # If fewer results were returned than the limit, no need to continue
        if result_count < N:
            break
        
        # Update the offset for the next iteration
        offset += N


# Nombre de tuples que l'utilisateur souhaite lire
a_lire = 20

lu = 0
for a in traiteRequete(requete, 10, a_lire):
  print(a)
  lu += 1
  if(lu >= a_lire):
    break

print()
if(lu < a_lire):
  print(f"Attention !!!! l'utilisateur n'a obtenu que {lu} lignes sur les {a_lire}")
else:
  print(f"OK, l'utilisateur a obtenu {lu} lignes sur les {a_lire}")

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Virtuoso 37000 Error SP030: SPARQL compiler, line 24: syntax error at \'OFFSET\' before \'0\'\n\nSPARQL query:\ndefine sql:big-data-const 0\n#output-format:application/sparql-results+json\nDEFINE sql:describe-mode "CBD"\n\nPREFIX dc:   <http://purl.org/dc/terms/>\nPREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>\n\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nPREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n\nPREFIX dbo:  <http://dbpedia.org/ontology/>\n\n# Dans cette source, les noms de propri\xc3\xa9t\xc3\xa9s sont en fran\xc3\xa7ais\nPREFIX dbp:  <http://fr.dbpedia.org/property/>\n\nSELECT distinct ?name ?p\nWHERE {\n  ?p rdf:type dbo:Person .\n  ?p foaf:name ?name\n  FILTER(  STRLEN(?name) != 0)\n}\nOFFSET 0 LIMIT 10\n OFFSET 0 LIMIT 10'

3) Proposer une solution pour accéder de manière progressive au résultat d'une requête de jointure entre 2 sites, par exemple J1.

In [ ]:
# à compléter


# ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT# ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
# # ! IMPORTANT
#  

# Exerice 4 : Cache de requêtes
1) Proposer des améliorations des requêtes précédentes afin d'éviter de poser plusieurs fois une même requête. On suppose que le résultat de la requête ne change pas entre deux invocations de la même requête.

2) Cache avec une capacité limitée : considérer qu'on peut conserver en cache le résultat de C requêtes (C=10) au maximum

In [ ]:
# A compléter

# FAQ: Questions diverses


## Mise au point des requêtes
Pour débugguer et **mettre au point vos requêtes** Sparql, il est utile de les tester directement sur l'interface du point d'accès dédié de la source de données. L'avantage est que cela vous retourne un message indiquant très précisément la nature et l'endroit où se trouve l'erreur dans la requête.
*   [BNF](http://data.bnf.fr/sparql) voir des [exemples](https://api.bnf.fr/sparql-endpoint-de-databnffr)
*   [DBPedia](http://fr.dbpedia.org/sparql) et la liste de tous les [prefixes](http://fr.dbpedia.org/sparql?nsdecl)


## Autres sources de données
Autres sources à votre disposition pour proposer des requêtes multi-sources:

1.   [Mondial](http://www.semwebtech.org/mondial/10/sparql) : données géopolitiques. Attention le résulat des requêtes est en XML pas en JSON.
2.   [Persée](data.persee.fr) : données bibliographiques alignées avec plusieurs autres sources dont dbpedia. Voir des [exemples](http://data.persee.fr/ressources/requetes-exemples/)
3.   [BioPortal](http://sparql.bioontology.org)
4.    [IdRef](https://data.idref.fr/sparql) serveur d’autorité IdRef et les références bibliographiques en provenance du Sudoc.
5.   [Insee](https://rdf.insee.fr/sparql)
6.  [Geonames web services](https://www.geonames.org/export/ws-overview.html)


## SPARKLIS : aide à l'écriture de requêtes Sparql
Pour exprimer vos requêtes en *langage naturel* et les traduire automatiquement en SPARQL, voir l'appli [Sparklis](http://www.irisa.fr/LIS/ferre/sparklis/)


*   onglet **Sparklis** pour exprimer vos requêtes en langage naturel
*   onglet **YASGUI** pour voir la traduction en Sparql



## Nombre d'éléments dans le resultat d'une requête
Pour compter le nombre d'éléments d'une requête, utiliser la notation avec requête imbriquée dans le where :

SELECT (count(1) as ?nb)

WHERE { *sous-requete* }
